In [116]:
from facenet_pytorch import InceptionResnetV1, MTCNN
import datetime
import time 
from mtcnn.detector import MtcnnDetector
import cv2
from align_faces import warp_and_crop_face, get_reference_facial_points
import numpy as np
from numpy.linalg import norm
import time

In [2]:

detector = MtcnnDetector()
def face_detection(img, output_size, align = True):
    """Nếu align == True thì sẽ có thêm bước Face Alignment sau khi nhận diện gương mặt,
        ngược lại thì không có thêm bước này"""
    
    # Nhận diện gương mặt
    boxes, facial5points = detector.detect_faces(img)
    
    # Tiến hành Alignment
    if align == True:
        if len(facial5points) != 0:
            facial5points = np.reshape(facial5points[0], (2, 5))

            default_square = True
            inner_padding_factor = 0.25
            outer_padding = (0, 0)

            ## Yêu cầu 10:
            ## VIẾT CODE Ở ĐÂY:

            # sử dụng hàm get_reference_facial_points
            reference_5pts = get_reference_facial_points(
                output_size, inner_padding_factor, outer_padding, default_square)
            
            # sử dụng hàm warp_and_crop_face
            face = warp_and_crop_face(img, facial5points, reference_pts = reference_5pts, crop_size = output_size)

    # Không Alignment
    else:
        (h,w) = img.shape[:2]
        if len(boxes) != 0:
            for box in boxes:
                (startX, startY, endX, endY) = box[:4].astype('int')
                (startX, startY) = (max(0, startX),max(0, startY))
                (endX, endY) = (min(w-1, endX), min(h-1, endY))
                face = img[startY:endY, startX:endX]
                face = cv2.resize(face, output_size)
                
    return face



In [4]:
def face_detect(image):
    
    img = image.copy()
    (h,w) = img.shape[:2]
    boxes, facial5points = detector.detect_faces(img)
    for box in boxes:
        (startX,startY,endX,endY)=box[:4].astype('int')

        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))

        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=img[startY:endY, startX:endX]
        face=cv2.resize(face,(160,160))

        # (mask,withoutMask) = model.predict(face.reshape(1,224,224,3))[0]

        # #determine the class label and color we will use to draw the bounding box and text
        # label='Mask' if mask>withoutMask else 'No Mask'
        # color=(0,255,0) if label=='Mask' else (0,0,255)
        # color=(0,255,0)
        #include the probability in the label
        # label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        # label = "Truyen"
        #display the label and bounding boxes
        # cv2.putText(img,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        # cv2.rectangle(img,(startX,startY),(endX,endY),color,2)
        
    return face

In [4]:
# res = InceptionResnetV1()

In [5]:
import glob
import torch 
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import os
from PIL import Image
import numpy as np

IMG_PATH = "Data/Image"
DATA_PATH = './data'
embeddings = []
names = []

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
def trans(img):
    transform = transforms.Compose([
            transforms.ToTensor(),
            fixed_image_standardization
        ])
    return transform(img).unsqueeze(0)
def fixed_image_standardization(image_tensor):
    processed_tensor = (image_tensor - 127.5) / 128.0
    return processed_tensor

In [ ]:
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

model.eval()

In [8]:
for usr in os.listdir(IMG_PATH):
    embeds = []
    for file in glob.glob(os.path.join(IMG_PATH, usr)+'/*.jpg'):
        try:
            # img = Image.open(file)
            img = cv2.imread(file)
            face = face_detect(img)
            cv2.imshow("face", face)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        except:
            continue
        with torch.no_grad():
            embed = model(trans(face).to(device))
            embeds.append(embed) #1 anh, kich thuoc [1,512]
    if len(embeds) == 0:
        continue
    embedding = torch.cat(embeds).mean(0, keepdim=True) #dua ra trung binh cua 50 anh, kich thuoc [1,512]
    embeddings.append(embedding.numpy()) # 1 cai list n cai [1,512]
    names.append(usr)

c:\Users\ASUS\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [20]:
print(names)
print(len(embeds))
# print(np.sqrt(512))

['Nam', 'Ngan', 'Truyen', 'Unknow1', 'Unknow2', 'Unknow3']
1


In [10]:
def embed_face(img):
    with torch.no_grad():
        embed = model(trans(img).to(device))
    return embed

In [11]:

img = cv2.imread("tocvang.JPG")
# img = cv2.resize(img,(480,720))
# img = cv2.resize(img,(128,128))
# out_putsize = (128,128)
face = face_detect(img)
eb_face = embed_face(face)
cv2.imshow("face",face)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [32]:
print(eb_face.shape)
len(embeddings)

torch.Size([1, 512])


12

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
def get_cosin(x, y):
    x = x.reshape(-1)
    y = y.reshape(-1)
    cos_sim = np.dot(y, x)/(norm(y)*norm(x))
    return cos_sim

y = eb_face 
def get_face(face_embed, names, embeddings):
    f = lambda x: get_cosin(x, y)
    list_cosin = [get_cosin(e1, face_embed) for e1 in embeddings]
    list_cosin2 = [cosine_similarity(face_embed, face_embed) for ex in embeddings]
    index_of_face = list_cosin.index(max(list_cosin))
    name = names[index_of_face]
    print(list_cosin2)
    return index_of_face, name, list_cosin
get_face(eb_face, names, embeddings)

[array([[1.]], dtype=float32), array([[1.]], dtype=float32), array([[1.]], dtype=float32), array([[1.]], dtype=float32), array([[1.]], dtype=float32), array([[1.]], dtype=float32)]


(2, 'Truyen', [0.9999999, 0.9999995, 1.0, 0.9999994, 0.9999994, 0.9999994])

In [14]:
print(names)

['Nam', 'Ngan', 'Truyen', 'Unknow1', 'Unknow2', 'Unknow3']


In [24]:
def face_recorinize(image):
    img = image.copy()
    (h,w) = img.shape[:2]
    boxes, facial5points = detector.detect_faces(img)
    for box in boxes:
        (startX,startY,endX,endY)=box[:4].astype('int')

        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))

        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=img[startY:endY, startX:endX]
        face=cv2.resize(face,(160,160))
        get_face()
        embed = embed_face(face)
        # (mask,withoutMask) = model.predict(face.reshape(1,224,224,3))[0]
        
        # #determine the class label and color we will use to draw the bounding box and text
        # label='Mask' if mask>withoutMask else 'No Mask'
        # color=(0,255,0) if label=='Mask' else (0,0,255)
        color=(0,255,0)
        #include the probability in the label
        # label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        label = "Truyen"
        index_of_face, name, list_cosin = get_face()
        #display the label and bounding boxes
        cv2.putText(img,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(img,(startX,startY),(endX,endY),color,2)
    return img

In [ ]:
video_capture = cv2.VideoCapture(id)
while True:
    x = time.time()
    ret, frame = video_capture.read()
    # img = frame[0:128,0:128]
    # print(model.predict(np.array([img])))
    img = face_recorinize(frame)
    print(time.time() - x)
    cv2.imshow('{}'.format(id), img)        
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()

torch.Size([1, 512])

In [145]:

print(names)
# print(embeddings)
data = np.asarray(embeddings).reshape(len(embeddings),512)
data

['Nam', 'Ngan', 'Truyen', 'Unknow1', 'Unknow2', 'Unknow3']


array([[-0.03484886,  0.0777184 ,  0.03608472, ...,  0.05145345,
         0.0110453 , -0.12757151],
       [-0.03486102,  0.07773694,  0.03610361, ...,  0.05148026,
         0.01106738, -0.12755129],
       [-0.03483398,  0.0777238 ,  0.03607829, ...,  0.05144887,
         0.01102702, -0.12760642],
       [-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
         0.01103554, -0.12758632],
       [-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
         0.01103554, -0.12758632],
       [-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
         0.01103554, -0.12758632]], dtype=float32)

In [141]:
data

array([[[-0.03484886,  0.0777184 ,  0.03608472, ...,  0.05145345,
          0.0110453 , -0.12757151]],

       [[-0.03486102,  0.07773694,  0.03610361, ...,  0.05148026,
          0.01106738, -0.12755129]],

       [[-0.03483398,  0.0777238 ,  0.03607829, ...,  0.05144887,
          0.01102702, -0.12760642]],

       [[-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
          0.01103554, -0.12758632]],

       [[-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
          0.01103554, -0.12758632]],

       [[-0.03484415,  0.07775436,  0.03613765, ...,  0.0514464 ,
          0.01103554, -0.12758632]]], dtype=float32)

In [143]:
print()

6


In [142]:
data = data.reshape(data.shape[0],512)
print(data)
print(data.shape)

[[-0.03484886  0.0777184   0.03608472 ...  0.05145345  0.0110453
  -0.12757151]
 [-0.03486102  0.07773694  0.03610361 ...  0.05148026  0.01106738
  -0.12755129]
 [-0.03483398  0.0777238   0.03607829 ...  0.05144887  0.01102702
  -0.12760642]
 [-0.03484415  0.07775436  0.03613765 ...  0.0514464   0.01103554
  -0.12758632]
 [-0.03484415  0.07775436  0.03613765 ...  0.0514464   0.01103554
  -0.12758632]
 [-0.03484415  0.07775436  0.03613765 ...  0.0514464   0.01103554
  -0.12758632]]
(6, 512)


In [120]:
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [125]:
label = np.array(le.fit_transform(names)).reshape(-1, 1)

print(label)
one_hot = OneHotEncoder(handle_unknown='ignore')
label = one_hot.fit_transform(label).toarray()
print(label)


[0 1 2 3 4 5]
[0 1 2 3 4 5]


In [ ]:
model1 = MLPClassifier(hidden_layer_sizes=(64),max_iter=100,verbose = True)
model1.fit(data, label)

In [66]:
label

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [119]:
s = le.fit_transform(names)
s.classes_

AttributeError: 'numpy.ndarray' object has no attribute 'classes_'

In [88]:
model2 = SVC(gamma='auto',probability = True)
model2.fit(data, le.fit_transform(names))

SVC(gamma='auto', probability=True)

In [94]:
pred = model2.predict_proba(data)
print(np.argmax(pred))

0


In [130]:
print(le.fit_transform(names))

[0 1 2 3 4 5]


In [131]:
print(data.shape)



(6, 512)


In [129]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',probability = True))
clf.fit(data,le.fit_transform(names))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', probability=True))])

In [109]:
1-clf.predict(data)

array([ 1,  0, -1, -4, -4, -4], dtype=int64)

In [110]:
import pickle
pickle.dump(clf,open("Model/stratify.pkl","wb"))
pickle.dump(le.fit_transform(names),open("List_user/lst_user.pkl","wb"))

In [ ]:
model3 = LinearSVC()

In [126]:
i_p = "sontung.jpg"
i = cv2.imread(i_p)

# img = cv2.resize(img,(480,720))
# img = cv2.resize(img,(128,128))
# out_putsize = (128,128)
face = face_detect(i)
eb_face = embed_face(face)
cv2.imshow("face",face)
cv2.waitKey(0)
cv2.destroyAllWindows()

c:\Users\ASUS\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [169]:
x = time.time()
out = 1 - clf.predict_proba(data)
print(np.max(out[0]))
print(time.time()-x)


0.9008256746882654
0.0020012855529785156


In [150]:
lst = pickle.load(open("List_user/2022_10_01_11-32-55_ListUser.pkl","rb"))
print(lst)

['Nam', 'Ngan', 'Truyen', 'Unknow1', 'Unknow2', 'Unknow3', 'Unknow4']
